# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "FTPMicrodadosNovoCaged"
extrator_bronze = AlinharETL(bucket,datamart)

2024-09-25 19:03:50,068 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [3]:
df = extrator_bronze.criar_view_temporaria('bronze/NovoCaged/MicrodadosNovoCagedSecao','compliance_secao')

2024-09-25 19:03:53,057 - INFO - Aguarde... Criando View (bronze/NovoCaged/MicrodadosNovoCagedSecao)
2024-09-25 19:04:00,922 - INFO - View criada com sucesso


# Tratamentos na tabela 

In [4]:
sql_query = """
    SELECT 
        CAST(`Código` AS STRING) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_secao
"""

In [5]:
df = extrator_bronze.carregar_dados_delta(sql_query)

2024-09-25 19:04:00,938 - INFO - Aguarde... Executando Query (Delta)
2024-09-25 19:04:00,940 - INFO - 
    SELECT 
        CAST(`Código` AS STRING) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_secao

2024-09-25 19:04:01,083 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [6]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/NovoCaged/MicrodadosNovoCagedSecao'

In [7]:
extrator_bronze.salvar_delta(df, 'overwrite')

2024-09-25 19:04:01,103 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 19:04:10,847 - INFO - Dados persistidos com sucesso
2024-09-25 19:04:10,848 - INFO - s3a://silver/NovoCaged/MicrodadosNovoCagedSecao
2024-09-25 19:04:10,849 - INFO - Aguarde... Realizando optimize
2024-09-25 19:04:13,952 - INFO - Optimize executado com sucesso.
2024-09-25 19:04:13,952 - INFO - Aguarde... Realizando vacuum
2024-09-25 19:04:40,940 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [8]:
extrator_bronze.parar_sessao()

2024-09-25 19:04:41,417 - INFO - Sessão Spark finalizada.
